In [ ]:
#in this script, once I get the list of percentage change in the given dataframe, 
#I drop all the rows where the percentage change is below 0
# obtained the selected rows, I request the capitalization of the stock, the p_e_ratio and revenue share

In [41]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import yfinance as yf
from datetime import date
from datetime import datetime
from pandas_datareader import data
import pandas_datareader as web
import datetime
import time
import math
import yahoo_fin.stock_info as si

In [2]:
#stock = pd.read_csv("stock.csv", compression='gzip', sep = ";")

In [161]:
stock = pd.read_parquet('stock_dat.parquet', engine='pyarrow')

In [144]:
#stock.bfill(axis = "rows", inplace = True)

In [213]:
#I get all the columns without the prefixe....please move this piece up in, we do not want "CLose_" anyway
stock.columns = stock.columns.str.lstrip('Close_') 

In [162]:
column_names = ["ticker", "pct"]

df = pd.DataFrame(columns = column_names)
df

,ticker,pct


In [164]:
#put the 
from_date = "2017-06-30"
to_date = "2022-06-30"

stock_pct = []
stock_dat = pd.DataFrame(stock_pct)

for i in stock.columns[1:]:
    pct = round((stock[i].loc[to_date]/stock[i].loc[from_date]-1)*100, 2)
    df_temp = [i, pct]
    #pd.concat(df, df_temp)
    df_length = len(df)
    df.loc[df_length] = df_temp

In [165]:
#convert nan into string
df["pct"] = df["pct"].map(lambda x: str(x))
#filter the rows where percentage is a value null or equal to 0
df = df.loc[(df["pct"] != "nan") & (df["pct"] != "0.0")]
#convert string into float
df["pct"] = df["pct"].map(lambda x: float(x))

C:\Users\jv3matzoli\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [167]:
df["ticker"] = df["ticker"].map(lambda x: x.replace("Close_", ""))

C:\Users\jv3matzoli\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [145]:
#I add a column with values of capitalization and p_e ratio

df["p_e_ratio"] = np.nan
df["capitalization"] = np.nan
df["revenue_pershare"] = np.nan
df['peg_ratio'] = np.nan
df['trailing_eps'] = np.nan

In [169]:
##I drop the columns of companies having a percentage increase below zero
df = df.drop(df[df.pct < 0].index)

In [170]:
df = df.reset_index(drop = True)

In [15]:
count= 0

for indice, elem in df.iterrows():
    try:

        ticker = elem["ticker"]
        #print(ticker)
        tick_analyzed = yf.Ticker(ticker)
        time.sleep(1)
        tick_analyzed_info = tick_analyzed.info
        #capitalization
        capit = int(web.get_quote_yahoo(ticker)['marketCap'].values)
        df.loc[indice, 'capitalization'] = capit
        #pe ratio
        pe_ratio = tick_analyzed_info["trailingPE"]
        df.loc[indice, 'p_e_ratio'] = pe_ratio
        #revenue per share
        revenuePerShare = tick_analyzed_info["revenuePerShare"]
        df.loc[indice, 'revenue_pershare'] = revenuePerShare
        #pegratio
        peg_ratio = tick_analyzed_info["pegRatio"]
        df.loc[indice, 'peg_ratio'] = peg_ratio
        #trailing EPS
        trailing_eps = tick_analyzed_info["trailingEps"]
        df.loc[indice, 'trailing_eps'] = trailing_eps

    
    except:
        pass
    

    count +=1
    if count % 100 == 0:
        print("We have reached the number:", count)

We have reached the number: 100
We have reached the number: 200
We have reached the number: 300
We have reached the number: 400
We have reached the number: 500
We have reached the number: 600
We have reached the number: 700
We have reached the number: 800
We have reached the number: 900
We have reached the number: 1000
We have reached the number: 1100
We have reached the number: 1200
We have reached the number: 1300
We have reached the number: 1400
We have reached the number: 1500
We have reached the number: 1600
We have reached the number: 1700
We have reached the number: 1800
We have reached the number: 1900
We have reached the number: 2000
We have reached the number: 2100
We have reached the number: 2200
We have reached the number: 2300
We have reached the number: 2400
We have reached the number: 2500
We have reached the number: 2600


In [171]:
#convert nan into string
df["pct"] = df["pct"].map(lambda x: str(x))
#filter the rows where percentage is a value null or equal to 0
df = df.loc[(df["pct"] != "nan") & (df["pct"] != "0.0")]
#convert string into float
df["pct"] = df["pct"].map(lambda x: float(x))


In [215]:
stock_order = stock.reset_index()

In [174]:
df_order_perc = df.sort_values(by = ["pct"], ascending = [False]).reset_index()

In [180]:
df_order_perc_filtered = df.merge(df_order_perc_filtered, how = "left", on ="ticker")

In [193]:
df_order_perc_filtered = df_order_perc_filtered.loc[(df_order_perc_filtered["pct"] > 0) 
                  & (df_order_perc_filtered["p_e_ratio"] >1) 
                  & (df_order_perc_filtered["p_e_ratio"] <200) 
                  & (df_order_perc_filtered["capitalization"] > 100000000)].sort_values(by = "pct", ascending = False).reset_index(drop = True)

In [203]:
df_order_perc_filtered

,ticker,pct,p_e_ratio,capitalization,revenue_pershare,peg_ratio,trailing_eps
0,DALXF,18654.87,6.250000,1.550210e+09,5.732,NaN,NaN
1,NYUKF,3660.00,1.584149,1.175105e+10,13501.862,NaN,NaN
2,KAKKF,3480.00,1.384570,6.568038e+09,8115.778,NaN,50.42
3,YACAF,2790.08,8.463415,4.886871e+09,4.096,NaN,NaN
4,FRNWF,1498.96,25.697496,2.608763e+09,6.138,NaN,0.84
...,...,...,...,...,...,...,...
1475,CGNX,0.16,28.045162,7.552391e+09,6.142,NaN,NaN
1476,SSMXY,0.13,39.726624,1.279969e+10,869.906,NaN,NaN
1477,STAR-PD,0.11,2.480392,7.606748e+08,5.243,NaN,NaN
1478,SXT,0.04,27.357141,3.380698e+09,32.781,NaN,NaN


In [120]:
#export some results
df_order_perc_filtered.to_excel("df_order_perc_filtered_4_07.xlsx", index = False)

In [230]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3
 
# Driver Code
intersection(list(df["ticker"]), (stock_order.columns))


['AMZN',
 'GOOG',
 'AA',
 'AXP',
 'ABT',
 'AMGN',
 'ALL',
 'ADBE',
 'GOOGL',
 'ABBV',
 'ADP',
 'ADSK',
 'ADM',
 'AMD',
 'AEP',
 'AVGO',
 'EA',
 'AEM',
 'APD',
 'AMBA',
 'NVS',
 'ANF',
 'LULU',
 'A',
 'AZO',
 'AN',
 'AZN',
 'AES',
 'BDX',
 'AB',
 'AFL',
 'ADI',
 'ACIW',
 'AMP',
 'AWK',
 'NVO',
 'ARCC',
 'AAP',
 'AIV',
 'AEE',
 'ANTM',
 'ABC',
 'ATI',
 'AVB',
 'ALNY',
 'LH',
 'AVY',
 'AUY',
 'ASH',
 'ABMD',
 'ABM',
 'ATO',
 'AON',
 'ADC',
 'ASML',
 'AMT',
 'ARW',
 'AR',
 'BAH',
 'AVT',
 'ALB',
 'AIZ',
 'SAIC',
 'AU',
 'APH',
 'JKHY',
 'AVAV',
 'AMKR',
 'AEIS',
 'Y',
 'AAON',
 'VRSK',
 'APO',
 'RBA',
 'MAA',
 'BAM',
 'ANET',
 'AIR',
 'RS',
 'PKG',
 'AVA',
 'ALSN',
 'ALGN',
 'AGO',
 'ROK',
 'GLPI',
 'ATRC',
 'ABB',
 'SEE',
 'PAAS',
 'AMED',
 'ADUS',
 'AAWW',
 'WMS',
 'PAG',
 'MDRX',
 'KLIC',
 'AMH',
 'AME',
 'AGCO',
 'WAL',
 'NOG',
 'ARE',
 'ANSS',
 'AIN',
 'ACLS',
 'ABG',
 'GPI',
 'AWR',
 'AWI',
 'RHHBY',
 'DOX',
 'AVID',
 'ASGN',
 'ACHC',
 'MANH',
 'LAMR',
 'ANGO',
 'WRLD',
 'UHAL',
 'SA

In [228]:
stock_order

,Date,BAC,AMZN,T,GOOG,MO,DAL,AA,AXP,DD,...,MAUSY,MEGGY,MEJHY,MET-PA,PLDGP,SAFLY,SEBYF,TEMPF,THKLY,
0,1999-12-31,25.093750,3.806250,36.820244,NaN,23.000000,NaN,99.724503,48.506207,63.394806,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-01-03,24.218750,4.468750,35.498489,NaN,23.437500,NaN,97.246407,45.880306,62.920383,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-01-04,22.781250,4.096875,33.421452,NaN,23.625000,NaN,97.696968,44.147945,61.200600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-01-05,23.031250,3.487500,33.940708,NaN,23.500000,NaN,103.329002,42.962643,63.187248,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-01-06,25.000000,3.278125,33.043808,NaN,23.062500,NaN,101.977310,43.837940,64.966331,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5678,2022-06-27,32.349998,113.220001,20.780001,2332.449951,43.189999,30.440001,49.560001,144.289993,57.570000,...,13.95,20.0,12.13,21.969999,64.0,2.9,108.934998,19.450001,9.62,31.41
5679,2022-06-28,32.259998,107.400002,20.610001,2251.429932,43.189999,30.330000,49.250000,142.190002,57.330002,...,13.95,20.0,12.40,21.820000,64.0,2.9,95.900002,19.450001,9.57,31.41
5680,2022-06-29,31.860001,108.919998,20.950001,2245.129883,41.759998,29.639999,47.619999,139.470001,56.330002,...,13.95,20.0,12.10,21.969999,64.0,2.9,95.900002,19.450001,9.38,31.41
5681,2022-06-30,31.129999,106.209999,20.959999,2187.449951,41.770000,28.969999,45.580002,138.619995,55.580002,...,13.95,20.0,12.10,21.719999,64.0,2.9,95.900002,19.450001,9.34,31.41


In [239]:
stock_order[intersection(list(df["ticker"]), (stock_order.columns))]

,AMZN,GOOG,AA,AXP,ABT,AMGN,ALL,ADBE,GOOGL,ABBV,...,BRKWF,BRTHF,PALAF,SECVY,GRP-UN,TREAF,HOCPF,MALRF,MALRY,MEGGY
0,3.806250,NaN,99.724503,48.506207,16.300713,60.062500,24.062500,16.812500,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.468750,NaN,97.246407,45.880306,15.711531,62.937500,23.500000,16.390625,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.096875,NaN,97.696968,44.147945,15.262630,58.125000,22.750000,15.015625,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.487500,NaN,103.329002,42.962643,15.234574,60.125000,23.750000,15.312500,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.278125,NaN,101.977310,43.837940,15.767643,61.125000,23.625000,15.437500,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5678,113.220001,2332.449951,49.560001,144.289993,109.080002,244.850006,126.190002,381.070007,2316.669922,153.139999,...,17.4,18.58,0.43,16.559999,61.430000,13.64,92.029999,32.84,33.500000,20.0
5679,107.400002,2251.429932,49.250000,142.190002,106.970001,243.509995,125.699997,365.630005,2240.149902,152.490005,...,17.4,18.58,0.42,16.559999,61.130001,13.64,86.000000,32.84,33.279999,20.0
5680,108.919998,2245.129883,47.619999,139.470001,109.099998,245.429993,125.419998,368.500000,2234.030029,154.139999,...,17.4,18.58,0.40,16.559999,60.389999,13.64,87.900002,32.84,33.279999,20.0
5681,106.209999,2187.449951,45.580002,138.619995,108.650002,243.300003,126.730003,366.059998,2179.260010,153.160004,...,17.4,18.58,0.40,16.559999,61.119999,13.64,85.290001,32.84,33.410000,20.0


In [240]:
df["ticker"]

0        AMZN
1        GOOG
2          AA
3         AXP
4         ABT
        ...  
2380    HOCPF
2381    MALRF
2382    MALRY
2383    MEGGY
2384    Close
Name: ticker, Length: 2385, dtype: object

In [236]:
len(intersection(list(df["ticker"]), (stock_order.columns)))

2125

In [235]:
stock_order_filter.shape, df.shape

((5683, 2156), (2385, 3))

In [233]:
##calculating coefficients
#we create a null columns
df["coef"] = np.nan

for indice, i in df.iterrows():
    ticker = i["ticker"]
    
    x = stock_order_filter.index.values.reshape(-1, 1)
    y = stock_order_filter[ticker].values.reshape(-1, 1)
    x = np.nan_to_num(x)
    y = np.nan_to_num(y)

    reg = LinearRegression().fit(x, y)
    coef = round(float(reg.coef_*100), 2)
    #print(coef)
    df.loc[df.index[indice], 'coef'] = coef


ValueError: Found input variables with inconsistent numbers of samples: [5683, 11366]

In [212]:
stock_order["Close_BAC"].values.reshape(-1, 1)

array([[25.09375   ],
       [24.21875   ],
       [22.78125   ],
       ...,
       [31.86000061],
       [31.12999916],
       [31.55999947]])

In [636]:
###df_order_coef = df.sort_values(by = "coef", ascending = False)

In [587]:
df_order_coef[:20]

,ticker,pct,coef
757,Close_ASCK,NaN,253495.57
3912,Close_FDBL,-100.00,5902.70
917,Close_BRK-A,242.54,3925.96
4121,Close_MWG,NaN,1359.36
58,Close_GAS,NaN,1351.70
1951,Close_NCT,NaN,1207.66
2382,Close_TNXP,NaN,937.42
3680,Close_FPT,NaN,788.54
2166,Close_NVIV,NaN,740.33
750,Close_CITY,-17.50,710.71
